In [1]:
import pandas as pd
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import logging


from graph import TransportGraph, ContactionTransportGraph
from forward_search import FCH
from dijkstra import Dijkstra

# Select city

In [60]:
CITY = 'brisbane'

## Download Transport Graphs

In [121]:
airport_codes = {
    'kuopio': 'KUO',
    'rennes': 'RNS',
    'venice': 'VCE',
    'belfast': 'BFS',
    'turku': 'TKU',
    'grenoble': 'GNB',
    'canberra': 'CBR',
    'luxembourg': 'LUX',
    'palermo': 'PMO',
    'nantes': 'NTE',
    'detroit': 'DTW',
    'toulouse': 'TLS',
    'bordeaux': 'BOD',
    'winnipeg': 'YWG',
    'dublin': 'DUB',
    'adelaide': 'ADL',
    'brisbane': 'BNE',
    'lisbon': 'LIS',
    'prague': 'PRG',
    'helsinki': 'HEL',
    'berlin': 'BER',
    'rome': 'FCO',
    'melbourne': 'MEL',
    'sydney': 'SYD',
}

In [ ]:
for city, code in airport_codes.items()

In [61]:
ttn_fc = pickle.load(open(F'graph_limited/{CITY}/{CITY}_tg_ttn_fc.pkl', 'rb'))

In [62]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
transport_connections = transport_connections[
    transport_connections['from_stop_I']!=transport_connections['to_stop_I']]

# Download walking connections
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

#Additional limit 
walk_connections = walk_connections[walk_connections['d_walk'] < 600]

#Build tg graph
ttn_fc_new = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)


In [63]:
ttn_fc_new.fractional_cascading_precomputation()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9801/9801 [00:00<00:00, 14877.12it/s]


In [69]:
np.mean([len(v[-1]) for (x, v) in ttn_fc_new.m_arr_fractional.items() if v])

36.251459549624684

In [70]:
np.mean([len(v[-1]) for (x, v) in ttn_fc.m_arr_fractional.items() if v])

29.839658048373646

In [102]:
np.mean([len(v[0]) for (x, v) in ttn_fc_new.m_arr_fractional.items() if v])

34.2708507089241

In [103]:
np.mean([len(v[0]) for (x, v) in ttn_fc.m_arr_fractional.items() if v])

38.3216221851543

# Calulate TTN for TG AND CH-graph

# Compare solutions

In [72]:
N = 1000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(ttn_fc.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(ttn_fc.nodes, 1)[0] for i in range(N)]
             })

algorithms = ['dijkstra_fractional', 'dijkstra_fractional_new']
duration = {x: [] for x in algorithms}
arrival = {x: 0 for x in algorithms}
indexes = []
for index, row in test_data.iterrows():
    random.shuffle(algorithms)
    for algorithm in algorithms:
        if algorithm == 'dijkstra_fractional':
            pathfinding = Dijkstra(graph=ttn_fc,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_fractional_new':
            pathfinding = Dijkstra(graph=ttn_fc_new,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
                indexes.append(index)
            arrival[algorithm] = path['arrival']
    arr = arrival[algorithms[0]]        
    for i in range(1, len(algorithms)):
        assert arr == arrival[algorithms[i]]  

/var/folders/kt/g2cs84fs4j3c1psy_vtfg4h40000gn/T/ipykernel_25967/1532740001.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(ttn_fc.nodes, 1)[0] for i in range(N)],
/var/folders/kt/g2cs84fs4j3c1psy_vtfg4h40000gn/T/ipykernel_25967/1532740001.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(ttn_fc.nodes, 1)[0] for i in range(N)]


In [39]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

dijkstra_fractional_new 34.01207729468599 19.97250160401866
dijkstra_fractional 34.97222222222222 70.66468722541403


In [25]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

dijkstra_fractional 73.76847826086957 80.57175903631526
dijkstra_fractional_new 84.62391304347825 96.94653319088604


In [48]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

dijkstra_fractional 73.08324324324325 76.76977579730098
dijkstra_fractional_new 88.06270270270271 94.51250947228866


In [57]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

dijkstra_fractional 71.53813104189044 81.90375397696256
dijkstra_fractional_new 83.40064446831364 88.53130039080239


In [74]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

dijkstra_fractional_new 115.94495412844037 119.4681512863863
dijkstra_fractional 101.72935779816514 114.47094162960848


In [111]:
from typing import Union, Dict, List
import heapdict
import time
import math
import logging
from bisect import bisect_left
# from binary_search import bisect_left

from graph import TransportGraph
from algorithms_wrapper import _check_running_time
from utils import to_milliseconds
from atf import ATF


class Dijkstra:
    def __init__(self, graph: TransportGraph, start_time: int, start_node: int, end_node: int):
        """
        Realization of Dijkstra algorithm
        https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm

        :param graph: TransportGraph Multimodal transport network
        :param start_time: int Start time in unix
        :param start_node: int Start node from, which we build a path
        :param end_node: int Target node
        """
        self.graph = graph

        self.source = start_node
        self.target = end_node
        self.start_time = start_time

        self.candidate_weights = {self.source: start_time}
        self.candidate_priorities = heapdict.heapdict({self.source: start_time})
        self.candidate_sequences = {self.source: [self.source]}
        self.candidate_route_names = {self.source: []}
        self.candidate_roots = {self.source: [self.source]}

    def shortest_path(self,
                      duration: Union[float, None] = None,
                      optimized_binary_search: bool = True,
                      fractional_cascading: bool = False
                      ) -> Dict[str, Union[List[Union[int, str]], int]]:

        exception = None

        winner_node = self.source
        winner_weight = self.start_time
        if optimized_binary_search:
            if fractional_cascading:
                start_time = time.monotonic()
                while (winner_node != self.target) and (not exception):

                    exception = _check_running_time(start_time, duration, "Dijkstra")

                    m_arr = self.graph.m_arr_fractional.get(winner_node)
                    pointers = self.graph.pointers.get(winner_node)
                    reachable_nodes = self.graph.reachable_nodes.get(winner_node)
                    out = self.graph.graph.get(winner_node)
                    if pointers:
                        start_index, next_loc = pointers[0][bisect_left(m_arr[0], winner_weight)]

                        node = reachable_nodes[0]
                        self._update_vertex_with_node_index_fractional_cascading_bus_profile(winner_node, winner_weight,
                                                                                             out, node, start_index)

                        for i in range(1, len(m_arr)):
                            if winner_weight <= m_arr[i][next_loc - 1]:
                                start_index, next_loc = pointers[i][next_loc - 1]
                            else:
                                start_index, next_loc = pointers[i][next_loc]

                            node = reachable_nodes[i]
                            self._update_vertex_with_node_index_fractional_cascading_bus_profile(winner_node,
                                                                                                 winner_weight,
                                                                                                 out, node, start_index)
                    for node in self.graph.walking_nodes.get(winner_node, []):
                        self._update_vertex_with_node_index_fractional_cascading_walk_profile(winner_node,
                                                                                              winner_weight, out, node)

                    try:
                        winner_node, winner_weight = self.candidate_priorities.popitem()
                    except IndexError:
                        message = f"Target {self.target} not reachable from node {self.source}"
                        logging.warning(message)
                        return {
                            'path': [],
                            'routes': [],
                            'roots': [],
                            'arrival': math.inf,
                            'duration': to_milliseconds(time.monotonic() - start_time)
                        }
            else:
                start_time = time.monotonic()
                while (winner_node != self.target) and (not exception):

                    exception = _check_running_time(start_time, duration, "Dijkstra")

                    departure = bisect_left(self.graph.nodes_schedule[winner_node], winner_weight)
                    nodes_indexes = self.graph.position_in_edge[winner_node].get(departure) #{nodeA: start_index, nodeB: start_index}

                    for node, f in self.graph.graph[winner_node].items():
                        self._update_vertex_with_node_index(node, winner_node, winner_weight, nodes_indexes)

                    try:
                        winner_node, winner_weight = self.candidate_priorities.popitem()
                    except IndexError:
                        message = f"Target {self.target} not reachable from node {self.source}"
                        logging.warning(message)
                        return {
                            'path': [],
                            'routes': [],
                            'roots': [],
                            'arrival': math.inf,
                            'duration': to_milliseconds(time.monotonic() - start_time)
                        }
        else:
            start_time = time.monotonic()
            while (winner_node != self.target) and (not exception):

                exception = _check_running_time(start_time, duration, "Dijkstra")

                for node, f in self.graph.graph[winner_node].items():
                    self._update_vertex(node, winner_node, winner_weight, f)

                try:
                    winner_node, winner_weight = self.candidate_priorities.popitem()
                except IndexError:
                    message = f"Target {self.target} not reachable from node {self.source}"
                    logging.warning(message)
                    return {
                        'path': [],
                        'routes': [],
                        'roots': [],
                        'arrival': math.inf,
                        'duration': to_milliseconds(time.monotonic() - start_time)
                    }
        if exception:
            return {
                'path': self.candidate_sequences[winner_node],
                'routes': self.candidate_route_names[winner_node],
                'roots': self.candidate_roots[winner_node],
                'arrival': winner_weight,
                'duration': to_milliseconds(time.monotonic() - start_time)
            }

        return {
            'path': self.candidate_sequences[self.target],
            'routes': self.candidate_route_names[winner_node],
            'roots': self.candidate_roots[winner_node],
            'arrival': winner_weight,
            'duration': to_milliseconds(time.monotonic() - start_time)
        }

    def _update_vertex(self, node: int, winner_node: int, winner_weight: int, f: ATF):
        """
        Update vertex iteration in Dijkstra

        :param node: int Node information about which we update
        :param winner_node: int. Parent node from each we reach this node
        :param winner_weight: Time in unix at which we have been at winner_node
        :param f: ATF Function, which represent movement from winner_node to node
        :return:
        """
        new_weight, sequence_nodes, route_names = f.arrival(winner_weight)
        if node in self.candidate_weights.keys():
            if new_weight < self.candidate_weights[node]:
                self.candidate_weights[node] = new_weight
                self.candidate_priorities[node] = new_weight
                self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
                self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
                self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names
        elif new_weight != math.inf:
            self.candidate_weights[node] = new_weight
            self.candidate_priorities[node] = new_weight
            self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
            self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
            self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names

    def _update_vertex_with_node_index(self, node: int, winner_node: int, winner_weight: int,
                                       nodes_indexes: Dict[int, int]):
        """
        Update vertex in TTN mode

        :param node: int Node information about which we update
        :param winner_node: int. Parent node from each we reach this node
        :param winner_weight: Time in unix at which we have been at winner_node
        :param nodes_indexes: Dict[int, int] Dictionaries, where key values are nodes
                            and values is index in ATF Bus profile
        :return:
        """
        l = walk_time = math.inf
        sequence_nodes = []
        route_names = []
        f = self.graph.graph[winner_node][node]
        if nodes_indexes:
            start_index = nodes_indexes[node]
            if start_index < f.size:
                l = f.buses[start_index].a
                sequence_nodes = f.buses[start_index].nodes
                route_names = f.buses[start_index].route_names
        if f.walk:
            walk_time = winner_weight + f.walk.w
        if walk_time < l:
            new_weight, sequence_nodes, route_names = walk_time, f.walk.nodes, f.walk.route_names
        else:
            new_weight, sequence_nodes, route_names = l, sequence_nodes, route_names

        if node in self.candidate_weights.keys():
            if new_weight < self.candidate_weights[node]:
                self.candidate_weights[node] = new_weight
                self.candidate_priorities[node] = new_weight
                self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
                self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
                self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names
        elif new_weight != math.inf:
            self.candidate_weights[node] = new_weight
            self.candidate_priorities[node] = new_weight
            self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
            self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
            self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names

    def _update_vertex_with_node_index_fractional_cascading_bus_profile(self, winner_node: int, winner_weight: int, out,
                                                                        node, start_index):
        """
        Update vertex in TTN mode

        :param winner_node: int. Parent node from each we reach this node
        :param winner_weight: Time in unix at which we have been at winner_node
        :return:
        """
        f = out[node]
        l = walk_time = math.inf
        sequence_nodes = []
        route_names = []
        if start_index is not None:
            if start_index < f.size:
                l = f.buses[start_index].a
                sequence_nodes = f.buses[start_index].nodes
                route_names = f.buses[start_index].route_names
        if f.walk:
            walk_time = winner_weight + f.walk.w
        if walk_time < l:
            new_weight, sequence_nodes, route_names = walk_time, f.walk.nodes, f.walk.route_names
        else:
            new_weight, sequence_nodes, route_names = l, sequence_nodes, route_names

        if node in self.candidate_weights.keys():
            if new_weight < self.candidate_weights[node]:
                self.candidate_weights[node] = new_weight
                self.candidate_priorities[node] = new_weight
                self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
                self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
                self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names
        elif new_weight != math.inf:
            self.candidate_weights[node] = new_weight
            self.candidate_priorities[node] = new_weight
            self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
            self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
            self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names

    def _update_vertex_with_node_index_fractional_cascading_walk_profile(self, winner_node: int, winner_weight: int,
                                                                         out, node):
        """
        Update vertex in TTN mode

        :param winner_node: int. Parent node from each we reach this node
        :param winner_weight: Time in unix at which we have been at winner_node
        :return:
        """

        f = out[node]
        if f.walk:
            walk_time = winner_weight + f.walk.w
            new_weight, sequence_nodes, route_names = walk_time, f.walk.nodes, f.walk.route_names
            if node in self.candidate_weights.keys():
                if new_weight < self.candidate_weights[node]:
                    self.candidate_weights[node] = new_weight
                    self.candidate_priorities[node] = new_weight
                    self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
                    self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
                    self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names
            elif new_weight != math.inf:
                self.candidate_weights[node] = new_weight
                self.candidate_priorities[node] = new_weight
                self.candidate_sequences[node] = self.candidate_sequences[winner_node] + sequence_nodes[1:]
                self.candidate_roots[node] = self.candidate_roots[winner_node] + [node]
                self.candidate_route_names[node] = self.candidate_route_names[winner_node] + route_names



In [119]:
row = test_data.iloc[indexes[356]]

pathfinding = Dijkstra(graph=ttn_fc_new,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
cProfile.run('''path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)''')
path['duration']

         419368 function calls (409890 primitive calls) in 0.551 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     7091    0.069    0.000    0.079    0.000 1943761160.py:222(_update_vertex_with_node_index_fractional_cascading_bus_profile)
    68443    0.227    0.000    0.264    0.000 1943761160.py:261(_update_vertex_with_node_index_fractional_cascading_walk_profile)
        1    0.089    0.089    0.551    0.551 1943761160.py:38(shortest_path)
        1    0.000    0.000    0.551    0.551 <string>:1(<module>)
     3275    0.001    0.000    0.024    0.000 _collections_abc.py:900(pop)
     5651    0.001    0.000    0.002    0.000 algorithms_wrapper.py:6(_check_running_time)
     9434    0.001    0.000    0.001    0.000 heapdict.py:105(__len__)
     9434    0.008    0.000    0.040    0.000 heapdict.py:30(__setitem__)
     8926    0.025    0.000    0.034    0.000 heapdict.py:39(_min_heapify)
     9434    0.003    0.000    0.0

551

In [120]:
row = test_data.iloc[indexes[356]]

pathfinding = Dijkstra(graph=ttn_fc,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
cProfile.run('''path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)''')
path['duration']

         419359 function calls (409874 primitive calls) in 0.317 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     7091    0.055    0.000    0.062    0.000 1943761160.py:222(_update_vertex_with_node_index_fractional_cascading_bus_profile)
    68443    0.091    0.000    0.127    0.000 1943761160.py:261(_update_vertex_with_node_index_fractional_cascading_walk_profile)
        1    0.039    0.039    0.317    0.317 1943761160.py:38(shortest_path)
        1    0.000    0.000    0.317    0.317 <string>:1(<module>)
     3274    0.001    0.000    0.024    0.000 _collections_abc.py:900(pop)
     5651    0.001    0.000    0.002    0.000 algorithms_wrapper.py:6(_check_running_time)
     9433    0.001    0.000    0.001    0.000 heapdict.py:105(__len__)
     9433    0.007    0.000    0.038    0.000 heapdict.py:30(__setitem__)
     8925    0.025    0.000    0.034    0.000 heapdict.py:39(_min_heapify)
     9433    0.003    0.000    0.0

316

In [118]:
path

{'path': [7985,
  7998,
  8002,
  8005,
  8007,
  7995,
  7308,
  7286,
  7284,
  11698,
  11696,
  11694,
  11692,
  11689,
  11686,
  11683,
  11680,
  11677,
  11674,
  11671,
  11669,
  11666,
  11664,
  11663,
  11662,
  11661,
  11659,
  11656,
  2215,
  712,
  700,
  702,
  12641,
  12615,
  12618,
  12621,
  12623,
  12625,
  12594,
  2289,
  2285,
  1820,
  1817,
  1812],
 'routes': ['walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  '618',
  'walk',
  '158',
  '263',
  '263',
  '263',
  'walk',
  'walk',
  '311',
  '311',
  '311',
  '311',
  '267',
  '267',
  'walk',
  'walk',
  'walk'],
 'roots': [7985,
  7998,
  8002,
  8005,
  8007,
  7995,
  7308,
  7286,
  7284,
  11698,
  11696,
  11694,
  11692,
  11689,
  11686,
  11683,
  11680,
  11677,
  11674,
  11671,
  11669,
  11666,
  116

In [77]:
row = test_data.iloc[414]
np.argmax([duration['dijkstra_fractional_new'][i] - v 
           for i, v in enumerate(duration['dijkstra_fractional'])])

356

In [78]:
duration['dijkstra_fractional_new'][356]

1206

In [79]:
duration['dijkstra_fractional'][356]

105

In [ ]:
pathfinding = Dijkstra(graph=ttn_fc,
          start_time=row['start_time'],
          start_node=row['start_node'], 
          end_node=row['end_node'])
path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)

In [18]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.8921973484025214,
   'duration': 50.07529411764706}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.892414692458161,
   'duration': 39.18016194331984}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9189306672462507,
   'duration': 21.63905325443787}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2058248206911542,
   'duration': 7.95}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.7315800912845034,
   'duration': 3.4444444444444446}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.4379482721147577,
   'duration': 0.0}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.8921973484025214,
   'duration': 42.74}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.892414692458161,
   'duration': 31.910931174089068}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9189306672462507,
   'duration': 18.5207100591716}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2058248206911542,
   'duration': 6.75}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.7315800912845034,
   'duration': 2.7777777777777777}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.4379482721147577,
   'duration': 0.0}]]

In [19]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [26]:
data = [[{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 7.732262382864793}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 12.339479392624728}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 9.299031476997579}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 8.369509043927648}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 7.720214190093708}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 12.339479392624728}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 6.824915824915825}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 7.9299719887955185}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 9.70875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 7.928571428571429}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 8.87875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 7.467065868263473}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 7.471556886227545}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 11.436008676789587}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 10.95906432748538}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 10.150121065375302}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 11.425162689804772}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 9.142118863049095}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 10.2439293598234}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 9.130490956072352}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 6.314814814814815}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 7.329131652661064}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 11.42407809110629}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 9.126614987080103}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 11.15121412803532}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 8.437751004016064}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 8.44578313253012}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 10.148910411622277}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 8.37467700258398}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 6.826599326599327}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 7.310924369747899}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 8.8375}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 6.318181818181818}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 11.164459161147903}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 10.219646799116997}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 10.164648910411623}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 10.971929824561403}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 7.917366946778712}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 7.312324929971989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 9.69875}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 9.70875}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 7.720214190093708}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 8.452476572958501}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 9.301452784503631}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 8.8475}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 10.237306843267108}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 6.877245508982036}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 8.381136950904393}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 10.03625730994152}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 10.95906432748538}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 11.147902869757175}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 10.024561403508772}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 12.366594360086768}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 6.883233532934132}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 9.297820823244551}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 10.024561403508772}]]

In [27]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [22]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.8921973484025214}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.892414692458161}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9189306672462507}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2058248206911542}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.7315800912845034}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.4379482721147577}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.8921973484025214}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.892414692458161}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9189306672462507}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2058248206911542}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.7315800912845034}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.4379482721147577}]]
for d_ in data:
    d = d_[0]
    if d['algorithm']=='dijkstra':
        d['duration'] = d['average_outgoing_edges']*np.log(56)
    else:
        d['duration'] = d['average_outgoing_edges'] + np.log(d['average_outgoing_edges']) + np.log(56)

In [25]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [17]:
import plotly.graph_objects as go

# Sample data
x = [3.8921973484025214, 2.892414692458161, 1.9189306672462507, 1.2058248206911542, 0.7315800912845034]
y = [99.9, 38.3, 19.5, 9.6, 4.63]

# Create a trace
trace = go.Scatter(x=x, y=y, mode='markers')

# Create a figure and add the trace
fig = go.Figure(trace)

# Show the plot
fig.show()

In [28]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 7.263466042154567}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 4.498002663115845}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 6.871446229913474}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 5.325526932084309}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 4.484687083888149}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 5.398843930635838}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 5.48953488372093}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.202185792349727,
   'duration': 7.26463700234192}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.1020036429872495,
   'duration': 3.9927745664739884}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 4.767352185089974}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 3.836115326251897}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 3.4725806451612904}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 4.272851296043656}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 3.8088012139605465}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 5.855263157894737}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 4.286493860845839}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 6.596401028277635}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.870673952641166,
   'duration': 4.593548387096774}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.214936247723132,
   'duration': 4.932014833127318}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.18421052631579}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 5.795361527967258}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.0856102003642987,
   'duration': 5.795361527967258}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.371584699453551,
   'duration': 5.796725784447476}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 6.292942743009321}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 3.9739884393063583}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 4.760925449871466}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.5154826958105647,
   'duration': 4.285129604365621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 5.404624277456647}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.9836065573770494,
   'duration': 6.1171770972037285}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.847587719298246}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.236794171220401,
   'duration': 6.592544987146529}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.2021857923497268,
   'duration': 6.11051930758988}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 7.627906976744186}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 7.2716627634660425}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.489981785063753,
   'duration': 5.401734104046243}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 7.638372093023256}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.726775956284153,
   'duration': 4.91350531107739}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.38615664845173,
   'duration': 5.472093023255814}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.515482695810564,
   'duration': 6.571979434447301}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.854280510018215,
   'duration': 5.467441860465116}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 0.9854280510018215,
   'duration': 3.995664739884393}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 4.582258064516129}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 8.179824561403509}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.278688524590164,
   'duration': 6.113182423435419}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 6.860321384425216}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.395264116575592,
   'duration': 3.456451612903226}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.19672131147541,
   'duration': 7.636046511627907}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.701275045537341,
   'duration': 5.322014051522248}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.098360655737705,
   'duration': 5.843201754385965}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.111111111111111,
   'duration': 8.18311403508772}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 4.930778739184178}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.3224043715846994,
   'duration': 4.76478149100257}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.6429872495446265,
   'duration': 5.327868852459017}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.666666666666666,
   'duration': 4.922610015174507}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.46448087431694,
   'duration': 4.953028430160693}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.080145719489982,
   'duration': 6.852904820766378}]]

In [29]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [30]:
data = [[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.183189655172415}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 4.08852867830424}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 4.595061728395062}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 5.2734375}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 7.788104089219331}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 4.826322930800543}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 7.328125}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.807112068965517}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 6.8190255220417635}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 4.597530864197531}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 4.909512761020881}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 6.8294663573085845}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 5.895910780669145}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 8.181034482758621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 6.345679012345679}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 3.3314527503526095}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 4.084788029925187}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 5.293526785714286}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 4.291201982651796}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 5.576059850374065}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 5.897149938042132}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 7.332589285714286}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 6.334567901234568}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 4.5851851851851855}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 6.346913580246913}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 3.5603799185888736}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 5.583603020496224}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 7.425646551724138}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 4.302354399008674}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 3.757105943152455}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 6.816705336426915}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 7.797195253505933}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 7.330357142857143}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 5.277901785714286}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 3.57123473541384}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 4.916473317865429}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 4.4724964739069115}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 4.087281795511222}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 3.757105943152455}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 5.103359173126615}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 4.302354399008674}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.109819121447028}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 7.793959007551241}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 5.590075512405609}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 5.8125}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 5.103359173126615}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 5.586839266450917}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 5.562344139650873}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 3.748062015503876}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 4.931554524361949}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 8.181034482758621}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 4.4724964739069115}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 4.815468113975577}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 5.327221438645981}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 5.574812967581047}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 7.788565264293419}]]

In [31]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()

In [32]:
data=[[{'algorithm': 'dijkstra',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 5.001}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 3.915}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 5.456}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 5.031}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 7.637}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 4.349}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 4.676}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 5.623}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 5.597}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 8.031}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 4.354}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 7.3843351548269585,
   'duration': 5.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.2003642987249545,
   'duration': 5.992}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 10.48816029143898,
   'duration': 5.323}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 5.989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 6.505}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 6.524}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.6411657559198543,
   'duration': 7.632}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.012}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 11.369763205828779,
   'duration': 4.147}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 3.197}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 5.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 12.276867030965391,
   'duration': 5.982}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 8.043}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 5.989}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 5.007}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.4626593806921675,
   'duration': 6.945}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 16.194899817850636,
   'duration': 5.722}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.868852459016393,
   'duration': 3.479}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.8524590163934427,
   'duration': 8.045}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 5.339}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 5.348}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 5.609}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 7.63}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 9.664845173041895,
   'duration': 3.73}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 6.955}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 3.096539162112933,
   'duration': 4.258}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.3934426229508197,
   'duration': 3.483}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 13.234972677595628,
   'duration': 4.706}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.699453551912568,
   'duration': 5.46}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 4.136}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 2.3205828779599273,
   'duration': 4.72}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 6.0783242258652095,
   'duration': 5.018}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 6.511}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.100182149362477,
   'duration': 3.914}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 5.0}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.8907103825136613,
   'duration': 5.142}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 2.083788706739526,
   'duration': 5.614}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 15.200364298724955,
   'duration': 6.78}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 1.9836065573770492,
   'duration': 3.907}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 8.109289617486338,
   'duration': 3.189}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.5136612021857925,
   'duration': 4.153}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 4.981785063752277,
   'duration': 4.344}],
 [{'algorithm': 'dijkstra',
   'average_outgoing_edges': 14.21311475409836,
   'duration': 6.949}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 5.5136612021857925,
   'duration': 4.711}],
 [{'algorithm': 'dijkstra_binary_duration',
   'average_outgoing_edges': 3.724954462659381,
   'duration': 3.724}]]


In [33]:
# Define colors for each algorithm
colors = {
    'dijkstra': 'blue',
    'dijkstra_binary_duration': 'red'
}

fig = go.Figure()

for sublist in data:
    for item in sublist:
        algorithm = item['algorithm']
        avg_outgoing_edges = item['average_outgoing_edges']
        duration = item['duration']
        fig.add_trace(go.Scatter(x=[avg_outgoing_edges], y=[duration], mode='markers', marker=dict(color=colors[algorithm]), name=algorithm))

fig.update_layout(title='Algorithm Performance',
                  xaxis_title='Average Outgoing Edges',
                  yaxis_title='Duration')

fig.show()


In [97]:
from numba import njit
from numba.typed import Dict
from numba.typed import List


@njit
def bisect_left(arr, x):
    lo = 0
    hi = len(arr)
    while lo < hi:
        mid = (lo + hi) // 2
        if arr[mid] < x:
            lo = mid + 1
        else:
            hi = mid
    return lo


@njit
def get_positions_fractional_cascading(m_arr, pointers, reachable_nodes, x, node):
    #locations = {}
    if pointers:
        loc, next_loc = pointers[0][bisect_left(m_arr[0], x)]
        #locations[reachable_nodes[node][0]] = loc
        for i in range(1, len(m_arr)):
            if x <= m_arr[i][next_loc - 1]:
                loc, next_loc = pointers[i][next_loc - 1]
            else:
                loc, next_loc = pointers[i][next_loc]
            #locations[reachable_nodes[node][i]] = loc
    return 0


In [99]:
get_positions_fractional_cascading(typed_m_arr_fractional, typed_pointers, tg.reachable_nodes[232], 1481541904, 229)

In [71]:
get_positions_fractional_cascading(tg.m_arr_fractional[232], tg.pointers[232], tg.reachable_nodes[232], 1481541904, 229)

In [17]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [38]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

# Results

In [ ]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))